In [1]:
import pandas as pd
import numpy as np
import editdistance as ed

In [2]:
Reactome_id = pd.read_excel('03_Reactome entrez genes ids.xlsx', header = None)
Kegg_id = pd.read_excel('05_KEGG entrez genes ids.xlsx', header = None)
Biocarta_id = pd.read_excel('07_BioCarta entrez genes ids.xlsx', header = None)

In [3]:
def rename_pathway(database,name):
    for j in range(len(database)):
        value = str(database.iloc[j][:1].values[0])
        symbol = value.replace(name,"")
        symbol = symbol.replace("_","")
        database = database.replace({value:symbol})
    return database

In [4]:
databases = [Reactome_id,Kegg_id,Biocarta_id]
names = ["REACTOME","KEGG","BIOCARTA"]

Reactome_id = rename_pathway(databases[0],names[0])
Kegg_id = rename_pathway(databases[1],names[1])
Biocarta_id = rename_pathway(databases[2],names[2])

In [34]:
same_symbol = []
for i in range(len(Reactome_id)):
    Reac_symbol = str(Reactome_id.iloc[i][:1].values[0])
    for j in range(len(Kegg_id)):
        Kegg_symbol = str(Kegg_id.iloc[j][:1].values[0])
        
        if Reac_symbol == Kegg_symbol:
            same_symbol.append([i,j])

In [35]:
same_symbol

[[9, 80],
 [42, 92],
 [234, 81],
 [235, 110],
 [265, 46],
 [288, 50],
 [372, 75],
 [503, 39],
 [678, 43],
 [708, 27],
 [979, 82],
 [1218, 2],
 [1386, 23],
 [1390, 7],
 [1485, 135]]

In [9]:
for i in range(len(same_symbol)):
    print(Reactome_id.iloc[same_symbol[i][0]][:1].values[0], Kegg_id.iloc[same_symbol[i][1]][:1].values[0])

BASEEXCISIONREPAIR BASEEXCISIONREPAIR
CELLCYCLE CELLCYCLE
NUCLEOTIDEEXCISIONREPAIR NUCLEOTIDEEXCISIONREPAIR
AXONGUIDANCE AXONGUIDANCE
SPHINGOLIPIDMETABOLISM SPHINGOLIPIDMETABOLISM
PYRUVATEMETABOLISM PYRUVATEMETABOLISM
DNAREPLICATION DNAREPLICATION
INOSITOLPHOSPHATEMETABOLISM INOSITOLPHOSPHATEMETABOLISM
ARACHIDONICACIDMETABOLISM ARACHIDONICACIDMETABOLISM
SELENOAMINOACIDMETABOLISM SELENOAMINOACIDMETABOLISM
MISMATCHREPAIR MISMATCHREPAIR
PENTOSEPHOSPHATEPATHWAY PENTOSEPHOSPHATEPATHWAY
PHENYLALANINEMETABOLISM PHENYLALANINEMETABOLISM
FATTYACIDMETABOLISM FATTYACIDMETABOLISM
LONGTERMPOTENTIATION LONGTERMPOTENTIATION


In [5]:
def edit_distance(database1,database2):
    all_similarity = dict()
    for i in range(len(database1)):
        first_symbol = str(database1.iloc[i][:1].values[0])
        first_sym_len = len(first_symbol)
        for j in range(len(database2)):
            second_symbol = str(database2.iloc[j][:1].values[0])
            second_sym_len = len(second_symbol)
            
            similarity = ed.distance(first_symbol,second_symbol)
            if first_sym_len > second_sym_len:
                per_similarity = round((1-(similarity/first_sym_len))*100,0)
                
            else:
                per_similarity = round((1-(similarity/second_sym_len))*100,0)
                
            if per_similarity in all_similarity:
                all_similarity[per_similarity] = [all_similarity.get(per_similarity),[i,j]]
            
            else:
                all_similarity[per_similarity] = [i,j]
    return all_similarity

In [84]:
bio_kegg_similar = edit_distance(Reactome_id,Biocarta_id)

In [85]:
dic_keys = list(bio_kegg_similar.keys())
dic_keys.sort()
dic_keys

[0.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 50.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 64.0,
 65.0,
 67.0,
 71.0,
 75.0,
 83.0]

In [89]:
print(bio_kegg_similar.get(75))

[[[[[[[[[[[[860, 19], [860, 44]], [860, 175]], [860, 179]], [860, 217]], [860, 235]], [860, 239]], [860, 243]], [860, 251]], [860, 266]], [860, 283]], [860, 286]]


In [87]:
similarity=[]
inter_uni=[]

In [93]:
x = Reactome_id.iloc[860].dropna()[3:].values
x = set(x)

y = Biocarta_id.iloc[179].dropna()[3:].values
y = set(y)
length_x = len(x)
length_y = len(y)

intersection = x & y
union = x | y
inter_uni.append([length_x,length_y,len(intersection),len(union)])
similary_per = round((len(intersection) / len(union)) * 100,0)
similarity.append(similary_per)

In [94]:
print(similarity)
print()
print(inter_uni)                               

[0.0, 2.0, 0.0, 2.0, 0.0]

[[91, 10, 0, 101], [91, 10, 2, 99], [91, 23, 0, 114], [91, 29, 2, 118], [91, 10, 0, 101]]


**combine Reactome and Kegg**

In [127]:
for i in range(len(same_symbol)):
    Reactome_id.drop(same_symbol[i][0], inplace= True)

In [128]:
Reac_Keg = pd.concat([Reactome_id,Kegg_id],axis = 0)
Reac_Keg.reset_index(drop = True, inplace = True)

In [129]:
rekg_bio = edit_distance(Reac_Keg,Biocarta_id)

rekg_keys = list(rekg_bio.keys())
rekg_keys.sort()

In [120]:

rekg_keys

[0.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 52.0,
 54.0,
 57.0]

In [85]:
print(rekg_bio.get(75))

print(str(Reac_Keg.iloc[850][:1].values[0]),
str(Biocarta_id.iloc[286][:1].values[0]))

[[[[[[[[[[[[850, 19], [850, 44]], [850, 175]], [850, 179]], [850, 217]], [850, 235]], [850, 239]], [850, 243]], [850, 251]], [850, 266]], [850, 283]], [850, 286]]
PCPCEPATHWAY PEPIPATHWAY


------------------

In [70]:
same_symbol2 = []
for i in range(len(Reac_Keg)):
    Reac_Kegg_symbol = str(Reac_Keg.iloc[i][:1].values[0])
    for j in range(len(Biocarta_id)):
        Bio_symbol = str(Biocarta_id.iloc[j][:1].values[0])
        
        if Reac_Kegg_symbol == Bio_symbol:
            same_symbol2.append([i,j])

In [71]:
same_symbol2

[]

In [136]:

print(rekg_keys)
print(rekg_bio.get(75))

[0.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 50.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 67.0, 71.0, 75.0, 83.0]
[[[[[[[[[[[[850, 19], [850, 44]], [850, 175]], [850, 179]], [850, 217]], [850, 235]], [850, 239]], [850, 243]], [850, 251]], [850, 266]], [850, 283]], [850, 286]]


In [134]:
similarity = []
inter_uni = []

In [91]:
similarity = []
inter_uni = []
for i in range(len(same_symbol)):
    x = Reactome_id.iloc[same_symbol[i][0]].dropna()[3:].values
    x = set(x)

    y = Kegg_id.iloc[same_symbol[i][1]].dropna()[3:].values
    y = set(y)

    intersection = x & y
    union = x | y
    inter_uni.append([len(intersection),len(union)])
    similary_per = round((len(intersection) / len(union)) * 100,0)
    similarity.append(similary_per)

In [92]:
similarity

[27.0,
 15.0,
 38.0,
 16.0,
 42.0,
 23.0,
 23.0,
 43.0,
 39.0,
 7.0,
 64.0,
 43.0,
 5.0,
 13.0,
 15.0]

In [93]:
inter_uni

[[26, 98],
 [102, 662],
 [42, 110],
 [93, 588],
 [37, 89],
 [13, 56],
 [30, 131],
 [30, 70],
 [32, 83],
 [9, 133],
 [14, 22],
 [12, 28],
 [1, 21],
 [25, 192],
 [12, 79]]